# The Basics of Cython

#### This notebook is a tutorial on the fundamental basics of Cython.

The most basic nature of Cython can be stated as: ___Cython is Python with C data types.___

There are several features of Cython that we need to be aware of:

1. ___Cython is Python:___ Almost any piece of Python code is also valid Cython code. 

2. The Cython compiler converts Python code into C code which makes equivalent calls ot the Python/C API (*this is partially how __Cython is Python__*).

3. Conversions of Python values and C values occurs automatically wherever possible. In addition to this, ___error checking___ of Python operations and ___reference counting___ are also done automatically.

4. Python's full power of exception handling is available in Cython, i.e., the `try`-except and `try`-finally statements, all in the midst of handling C data.

So, what we can expect is that we will be able to write our usual Python code but this time we can add an extra touch of flair to it with C data types. This will make our code run faster, especially when we are working with large datasets.

Without wasting too much time, let's get cracking!

***

## Our first progam: "Hello World!"


We'll start out by attempting to print `"Hello World!"`. To do this, we need several things:

1. the source file with the extension `.pyx`
2. the setup or compile file - an equivalent to 'makefile'

Our source file will have the following code:

```python
print('Hello World!')
```

while our setup file will have the following:

```python
from distutils.core import setup
from Cython.Build import cythonize

setup(ext_modules=cythonize('hello_world.pyx'))
```

We'll then use the following command to compile our code:

```bash
python3 setup.py build_ext --place
```

which will result in the creation of the `hello_world.so` file in unix or `hello_world.pyd` in Windows.

In [1]:
# Our 'hello_world.pyx' file - this is the file with the actual source
!touch hello_world.pyx
!echo "print('Hello World!')" >> hello_world.pyx

# Our 'setup.py' file - this is similar to a makefile but it's for Python
!touch setup.py
!echo "from distutils.core import setup" >> setup.py
!echo "from Cython.Build import cythonize" >> setup.py
!echo "setup(ext_modules=cythonize('hello_world.pyx'))" >> setup.py

In [4]:
# Now we can build the Cython file
!python3 setup.py build_ext --inplace
!ls

running build_ext
basics_of_cython.ipynb            hello_world.cpython-36m-darwin.so
build                             hello_world.pyx
hello_world.c                     setup.py


In [3]:
# Importing the 'hello_world.pyx' file - just like a regular Python module
import hello_world

Hello World!


Voila! We have just written and built our first Cython program.